In [1]:
import os
import pprint
from utils import secret_loader
import openai

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage, 
    HumanMessage, 
    SystemMessage
)
from langchain.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

### req
1. Installation
    ```bash
      pip install langchain
    ```

2. Setup API Key

    option 1. <bash> `setx OPENAI_API_KEY “<yourkey>”`  
    option 2. save in `secret.json` and add code below  
    ```py
    OPENAI = secret_loader.SecretLoader("OpenAI")  
    os.envrion["OPENAI_API_KEY"] = OPENAI["api_key"]
    ```

In [2]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
SERPER = secret_loader.SecretLoader("Serper")
os.environ["SERPAPI_API_KEY"] = SERPER["api_key"]

# LLMs

- default : text-davinci-003

In [4]:
llm = OpenAI(temperature=0.9)
    ## temperature : 높을수록 다양한 문장 생성
llm.predict("Name an A.I. chatbot that gives skin advice.")

'\n\n1. SkinGenius\n2. skINfluence\n3. Hera Skin Coach\n4. Skinenius\n5. Neutrogena Skin360 Chatbot'

# Chat models

- default : gpt-3.5-turbo

In [5]:
chat = ChatOpenAI(temperature=0)
chat.predict_messages([HumanMessage(content="맑고 투명하고 촉촉한 느낌의 피부 상담 챗봇을 만들었는데 이름을 지어주세요.")])

AIMessage(content='"피부 클리어링 챗봇"', additional_kwargs={}, example=False)

# Prompt templates

- LLM의 input으로 들어갈 메세지가 담긴 구조화된 틀
- chain이 실행되는 시점에 template을 format할 수 있는 input variables 설정 가능
- chain에 의해 memory와 연결된 경우 자동으로 해당 정보를 불러오기 가능  
  ⭐️ template 내 `memory_key`와 동일한 변수명을 가진 변수의 위치에 자동 mapping됨 

Chat model
- 메세지 프롬프트 : `SystemMessagePromptTemplate`, `HumanMessagePromptTemplate`
- 대화 프롬프트 : 연속된 메세지 프롬프트를 담는 `ChatPromptTemplate`
- 대화 프롬프트 format : `format_messages` or `format`
    - output type : `ChatPromptTemplate` or `str`
    
LLMs
- 텍스트 프롬프트 : `PromptTemplate`
- 텍스트 프롬프트 format : `format`
    - output type : `str`

## Chat models

In [2]:
template = "You are a dermatologist with years of experience in treating various skin conditions. You have extensive knowledge of skincare products and medical treatments for maintaining healthy skin. You are skilled at analyzing different skin types and characteristics, including texture, tone, and sensitivities. You can provide personalized advice based on an individual's unique skin needs. You can recommend appropriate skincare products for various skin concerns, such as acne, dryness, or sensitivity.\n\nWhen presented with a new question, please (1) provide a helpful initial response to the question and ask {question_num} follow-up questions for clarification, making it mandatory for the user to answer them. (2) Upon receiving answers to the follow-up questions, listen to the user's responses, and (3) offer a final, more informed answer that incorporates the additional information gathered."
human_template = "{user_question}" 

# from_template : string 내 input variables 파악, 해당 클래스의 인스턴스 생성
system_message_prompt = SystemMessagePromptTemplate.from_template(template) # input variable : question_num
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template) # input variable : user_question
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [5]:
chat_prompt.format_messages(question_num=3, user_question="Recommend a summer skincare routine.")

[SystemMessage(content="You are a dermatologist with years of experience in treating various skin conditions. You have extensive knowledge of skincare products and medical treatments for maintaining healthy skin. You are skilled at analyzing different skin types and characteristics, including texture, tone, and sensitivities. You can provide personalized advice based on an individual's unique skin needs. You can recommend appropriate skincare products for various skin concerns, such as acne, dryness, or sensitivity.\n\nWhen presented with a new question, please (1) provide a helpful initial response to the question and ask 3 follow-up questions for clarification, making it mandatory for the user to answer them. (2) Upon receiving answers to the follow-up questions, listen to the user's responses, and (3) offer a final, more informed answer that incorporates the additional information gathered.", additional_kwargs={}),
 HumanMessage(content='Recommend a summer skincare routine.', additi

## LLMs

In [8]:
prompt = PromptTemplate.from_template("Tell me what type of information you need as a dermatologist from a patient to advise them on {skin_concerns} of their skin concerns.")
prompt.format(skin_concerns="Narrow-headed acne")

'Tell me what type of information you need as a dermatologist from a patient to advise them on Narrow-headed acne of their skin concerns.'

```py
def format(self, **kwargs: Any) -> str:
    return self.format_prompt(**kwargs).to_string()
```

# Chains

- llm(`ChatOpenAI` or `OpenAI`)과 prompt template(`chat_prompt` or `prompt`)의 결합체를 의미
- 사용자의 목적에 따라 다양한 chain 구성 가능 (ex. `ConversationChain`, `RetrievalQA`)  
  ➡️ chain의 역할이 정해짐에 따라 이를 잘 수행할 수 있는 prompt 구성을 찾게 됨(=PromptEngineering)

## Chat models

In [9]:
chain_chat = ChatOpenAI(temperature=0)
chain = LLMChain(llm=chain_chat, prompt=chat_prompt) # chain = LLM + prompt
chain.run(question_num=3, user_question="Recommend a summer skincare routine.")

"A good summer skincare routine should focus on protecting your skin from the sun, keeping it hydrated, and preventing breakouts caused by sweat and excess oil. Here's a basic summer skincare routine to get you started:\n\n1. Cleanse: Start your routine by cleansing your face with a gentle cleanser. This will remove dirt, sweat, and excess oil without stripping your skin of its natural moisture. Do you have any specific skin concerns or sensitivities that I should consider when recommending a cleanser?\n\n2. Hydrate: After cleansing, apply a lightweight, oil-free moisturizer to keep your skin hydrated without feeling heavy or greasy. Look for a moisturizer with SPF to provide sun protection. Do you have any specific preferences or concerns when it comes to moisturizers?\n\n3. Sunscreen: Apply a broad-spectrum sunscreen with at least SPF 30 every morning, even on cloudy days. Reapply every two hours if you're spending time outdoors. Do you have any specific preferences or concerns when 

A good summer skincare routine should focus on protecting your skin from the sun, keeping it hydrated, and preventing breakouts caused by sweat and excess oil. Here's a basic summer skincare routine to get you started:

1. Cleanse: Start your routine by cleansing your face with a gentle cleanser. This will remove dirt, sweat, and excess oil without stripping your skin of its natural moisture. Do you have any specific skin concerns or sensitivities that I should consider when recommending a cleanser?

2. Hydrate: After cleansing, apply a lightweight, oil-free moisturizer to keep your skin hydrated without feeling heavy or greasy. Look for a moisturizer with SPF to provide sun protection. Do you have any specific preferences or concerns when it comes to moisturizers?

3. Sunscreen: Apply a broad-spectrum sunscreen with at least SPF 30 every morning, even on cloudy days. Reapply every two hours if you're spending time outdoors. Do you have any specific preferences or concerns when it comes to sunscreen?

4. Exfoliate: Use a gentle exfoliator once or twice a week to remove dead skin cells and unclog pores. This will help prevent breakouts and keep your skin looking fresh. Do you have any specific skin concerns or sensitivities that I should consider when recommending an exfoliator?

5. Hydrating face masks: Treat your skin to a hydrating face mask once a week to replenish moisture and soothe any sun-exposed skin. Look for masks with ingredients like aloe vera or hyaluronic acid. Do you have any specific preferences or concerns when it comes to face masks?

Please let me know your specific skin concerns, sensitivities, and preferences so that I can provide a more tailored recommendation for your summer skincare routine.

## LLMs

In [10]:
prompt = PromptTemplate.from_template("Tell me what type of information you need as a dermatologist from a patient to advise them on {skin_concerns} of their skin concerns.")
chain = LLMChain(llm=llm, prompt=prompt)
chain.run("Narrow-headed acne")

"\n\n1. Patient's medical history: To ascertain any underlying conditions that could be contributing to the acne.\n2. Current condition of skin: To determine the severity of the condition and the most appropriate treatment plan.\n3. Patient's lifestyle: To be able to assess any factors such as diet or stress that could be causing or exacerbating the condition.\n4. Patient's current skincare routine: To assess any products that may be contributing to the acne.\n5. Patient's age and gender: To tailor a treatment plan to meet the needs of the individual patient."

1. Patient's medical history: To ascertain any underlying conditions that could be contributing to the acne.
2. Current condition of skin: To determine the severity of the condition and the most appropriate treatment plan.
3. Patient's lifestyle: To be able to assess any factors such as diet or stress that could be causing or exacerbating the condition.
4. Patient's current skincare routine: To assess any products that may be contributing to the acne.
5. Patient's age and gender: To tailor a treatment plan to meet the needs of the individual patient.

# Memory

- 대화 생성처럼 동일한 chain을 반복하면서 이전 chain의 결과를 다음 chain의 prompt에 반영하려는 경우,  
  output 정보를 저장해주고 이를 prompt와 자동으로 연결시켜주는 모듈 필요(=`Memory`)
- memory는 자신이 속한 외부 chain의 output을 그대로 저장하거나(ex. 대화 생성 chain의 `ConversationBufferMemory`),  
  자신의 내부 chain을 한번 더 거쳐 가공된 형태로 저장(ex. 대화 생성 chain의 `ConversationSummaryMemory` : 내부에 요약을 위한 chain 존재)
- ❗️ 같은 chain을 반복하더라도 그 결과가 다음 chain에 영향을 주지 않는 경우, memory가 아닌 프로그램 변수로 관리해도 충분

## Chat models

In [13]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and "
                                              "provides lots of specific details from its context. If the AI does not know the answer to a "
                                              "question, it truthfully says it does not know."
    ),
    # MessagePlaceholder : format하기 전까지 ChatPromptTemplate에 어떤 Message가 들어갈지 모를 때 활용
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])
llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True) 
        # memory_key="history" ➡️ MessagesPlaceholder와 연결
        # return_messages=True : string이 아닌 Message type으로 return
# ConversationChain : default memory로 ConversationBufferMemory를 가지고 있음
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)
conversation.predict(input="Hi, how are you?")

"Hello! I'm an AI language model, so I don't have feelings, but I'm here to help you. How can I assist you today?"

In [14]:
conversation.predict(input="Suggest a recipe for a simple dish menu.")

"Certainly! How about a classic recipe for spaghetti aglio e olio? It's a simple and delicious Italian dish. Here's what you'll need:\n\nIngredients:\n- 8 ounces of spaghetti\n- 4 cloves of garlic, thinly sliced\n- 1/4 cup of extra virgin olive oil\n- 1/2 teaspoon of red pepper flakes (adjust to your spice preference)\n- Salt, to taste\n- Freshly ground black pepper, to taste\n- 1/4 cup of chopped fresh parsley\n- Grated Parmesan cheese (optional)\n\nInstructions:\n1. Cook the spaghetti according to the package instructions until al dente. Drain and set aside.\n2. In a large skillet, heat the olive oil over medium heat. Add the sliced garlic and red pepper flakes. Cook for about 2-3 minutes until the garlic turns golden brown and fragrant. Be careful not to burn the garlic.\n3. Add the cooked spaghetti to the skillet and toss well to coat the pasta with the garlic-infused oil. Season with salt and black pepper to taste.\n4. Remove the skillet from heat and sprinkle the chopped parsley 

Certainly! How about a classic recipe for spaghetti aglio e olio? It's a simple and delicious Italian dish. Here's what you'll need:

Ingredients:
- 8 ounces of spaghetti\n- 4 cloves of garlic, thinly sliced
- 1/4 cup of extra virgin olive oil
- 1/2 teaspoon of red pepper flakes (adjust to your spice preference)
- Salt, to taste
- Freshly ground black pepper, to taste
- 1/4 cup of chopped fresh parsley
- Grated Parmesan cheese (optional)

Instructions:
1. Cook the spaghetti according to the package instructions until al dente. Drain and set aside.
2. In a large skillet, heat the olive oil over medium heat. Add the sliced garlic and red pepper flakes. Cook for about 2-3 minutes until the garlic turns golden brown and fragrant. Be careful not to burn the garlic.
3. Add the cooked spaghetti to the skillet and toss well to coat the pasta with the garlic-infused oil. Season with salt and black pepper to taste.
4. Remove the skillet from heat and sprinkle the chopped parsley over the spaghetti. Toss again to combine.
5. Serve the spaghetti aglio e olio hot, garnished with grated Parmesan cheese if desired.

Enjoy your simple and flavorful spaghetti aglio e olio! Let me know if there's anything else I can assist you with.

In [15]:
conversation.predict(input="Give me alternatives when I don't have extra virgin olive oil.")

"If you don't have extra virgin olive oil on hand, don't worry! There are a few alternatives you can use in the spaghetti aglio e olio recipe:\n\n1. Regular Olive Oil: Regular olive oil is a suitable substitute for extra virgin olive oil in this recipe. It has a milder flavor but will still work well.\n\n2. Vegetable Oil: Vegetable oil, such as canola or sunflower oil, can be used as a substitute. Keep in mind that it has a neutral flavor, so the dish may have a slightly different taste.\n\n3. Butter: If you prefer a richer flavor, you can use butter instead of olive oil. Melt the butter in the skillet and sauté the garlic and red pepper flakes in it. Be aware that this will alter the traditional taste of the dish.\n\nRemember, the choice of oil or butter will affect the overall flavor of the dish, so choose an alternative that suits your taste preferences. Enjoy your spaghetti aglio e olio! Let me know if there's anything else I can help you with."

If you don't have extra virgin olive oil on hand, don't worry! There are a few alternatives you can use in the spaghetti aglio e olio recipe:

1. Regular Olive Oil: Regular olive oil is a suitable substitute for extra virgin olive oil in this recipe. It has a milder flavor but will still work well.

2. Vegetable Oil: Vegetable oil, such as canola or sunflower oil, can be used as a substitute. Keep in mind that it has a neutral flavor, so the dish may have a slightly different taste.

3. Butter: If you prefer a richer flavor, you can use butter instead of olive oil. Melt the butter in the skillet and sauté the garlic and red pepper flakes in it. Be aware that this will alter the traditional taste of the dish.

Remember, the choice of oil or butter will affect the overall flavor of the dish, so choose an alternative that suits your taste preferences. Enjoy your spaghetti aglio e olio! Let me know if there's anything else I can help you with.

## LLMs

In [16]:
llm = OpenAI(temperature=0)
# ConversationChain : default memory로 ConversationBufferMemory를 가지고 있음
conversation = ConversationChain(memory=memory, llm=llm, verbose=True)
conversation.run("Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. My name is AI. What's your name?"

In [17]:
conversation.run("I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. My name is AI. What's your name?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

# Agents

`initialize_agent`의 모델 파라미터 입력 차이
- tool 설정 가능 ➡️ prompt도 함께 설정됨

### Req

<bash>`pip install google-search-results`

## Chat models

In [11]:
chat = ChatOpenAI(temperature=0)
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
    ## Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
agent = initialize_agent(tools, chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
    ## ZERO_SHOT_REACT_DESCRIPTION : 0-shot learning을 통해 대화를 이어나가는 agent
    ## verbose : True일 경우, agent의 상태를 출력
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")



> Entering new AgentExecutor chain...
I need to find the high temperature in SF yesterday and then raise it to the .023 power.
Action: Search
Action Input: "high temperature in SF yesterday"
Observation: High: 69.8ºf @12:30 PM Low: 55.94ºf @3:56 AM Approx. Precipitation / Rain Total: in. 1hr.
Thought:I found the high temperature in SF yesterday, but I need to convert it to Fahrenheit and then raise it to the .023 power.
Action: Calculator
Action Input: (69.8 * 9/5) + 32
Observation: Answer: 157.64
Thought:I now have the high temperature in SF yesterday in Fahrenheit. Now I can raise it to the .023 power.
Action: Calculator
Action Input: 157.64 ^ 0.023
Observation: Answer: 1.1234308038113916
Thought:I now know the final answer.
Final Answer: The high temperature in SF yesterday in Fahrenheit raised to the .023 power is approximately 1.1234.

> Finished chain.


'The high temperature in SF yesterday in Fahrenheit raised to the .023 power is approximately 1.1234.'

## LLMs

In [12]:
llm = OpenAI(temperature=0)
tools = tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")



> Entering new AgentExecutor chain...
 I need to find the temperature first, then use the calculator to raise it to the .023 power.
Action: Search
Action Input: "High temperature in SF yesterday"
Observation: High: 69.8ºf @12:30 PM Low: 55.94ºf @3:56 AM Approx. Precipitation / Rain Total: in. 1hr.
Thought: I now need to use the calculator to raise 69.8 to the .023 power
Action: Calculator
Action Input: 69.8^.023
Observation: Answer: 1.1025763550530143
Thought: I now know the final answer
Final Answer: 1.1025763550530143

> Finished chain.


'1.1025763550530143'